In [6]:
import os
import glob
import subprocess
import json
import warnings
from typing import List, Union
from typing import Optional

import ee
import geemap
import rasterio
import sankee
import fiona
from osgeo import gdal, osr, gdalconst
import numpy.ma as ma

In [ ]:
from functions import *

In [3]:
#ee.Authenticate()

Enter verification code:  4/1AbUR2VNX6roYyX2FCncI3NKDRzsgg7PzCPjhNVUUvsDAsfjo-ZwFqlQJmZM



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [30]:
def scale_landsat(image):
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True)

# Define a function to clip each image to the feature collection
def clip_image(image):
    clipped_image = image.clip(site)
    return clipped_image

    
def download_collection_images_L8(collection_id, output_dir, start_date, end_date, cloud, site):
    # Load the Earth Engine collection
    collection = (ee.ImageCollection(collection_id)
                    .filterBounds(site)
                    .map(clip_image)
                    .filterDate(start_date, end_date)
                    .filterMetadata('CLOUD_COVER', 'less_than', cloud)
                    .map(scale_landsat)
                 )
                       

    # Get the image IDs
    image_ids = collection.aggregate_array('system:index').getInfo()

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through the image IDs and download each image
    for image_id in image_ids:
        image = ee.Image(collection.filter(ee.Filter.eq('system:index', image_id)).first()).select(['SR_B2', 'SR_B3' ,'SR_B4', 'SR_B5'])
        # Get the image download URL
        geemap.ee_export_image(
               image,
               filename=output_dir +'\\'+ image_id+'.tif',
               scale=30,
               region=site.geometry(),
               file_per_band=False
            )
        print(f"Downloaded {image_id}.tif")

In [31]:
site = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\ICoAST sites\\Sites\\Mandurah.shp'))

In [32]:
collection_id = 'LANDSAT/LC08/C02/T1_L2'
start_date = '2022-01-01'
end_date = '2022-12-30'
output_dir = os.path.join(os.getcwd(),'Data\\L8 Downloads\\2022')

In [33]:
download_collection_images_L8(collection_id, output_dir, start_date= start_date, end_date = end_date, cloud = 15, site = mandurah)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\Landsat-SAV\Data\L8 Downloads\2022\LC08_112083_20220116.tif
Downloaded LC08_112083_20220116.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\Landsat-SAV\Data\L8 Downloads\2022\LC08_112083_20220201.tif
Downloaded LC08_112083_20220201.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\Landsat-SAV\Data\L8 Downloads\2022\LC08_112083_20220217.tif
Downloaded LC08_112083_20220217.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\Landsat-SAV\Data\L8 Downloads\2022\LC08_112083_20220305.tif
Downloaded LC08_112083_20220305.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\Landsat-SAV\Data\L8 Downloads\2022\LC08_112083_20220406.tif
Downloaded LC08_112083_20220406.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\Landsat-SAV\Data\L8 Downloads\2022\LC08_112083_20220422.tif
Down